In [41]:
import json
import os
import pandas as pd
import re
import requests

from glob import glob
from time import sleep

## Loading telegram messages as a DataFrame

Before running the following code, make sure you have ran `main.py` a couple of times.

In [6]:
json_files = glob("*-messages.json")

In [7]:
def grab_wanted_object_properties(parsed_json):
    wanted_props = {
        "message_id": parsed_json['id'],
        "date": parsed_json['date'],
    }
    wanted_props["message"] = parsed_json['message'] if "message" in parsed_json else None
    return wanted_props

In [8]:
formatted_list = []
for file in json_files:
    with open(file, 'r') as json_file:
        loaded_file = json.load(json_file)
    for obj in loaded_file:
        formatted_list.append(grab_wanted_object_properties(obj))

In [11]:
df = pd.DataFrame(formatted_list)

In [12]:
df.head()

,message_id,date,message
0,66378,2023-03-04 22:40:51+00:00,https://youtu.be/CoJ23XNHgG0
1,66377,2023-03-04 22:32:39+00:00,https://youtu.be/kqNJaKRnuVg
2,66376,2023-03-04 22:28:51+00:00,Tu que lute chegado
3,66375,2023-03-04 22:28:28+00:00,Love sons
4,66374,2023-03-04 22:27:57+00:00,Rádio Mate.


## Create a new column for youtube urls found in the message body

In [13]:
def find_youtube_url(message):
    if message is None:
        return None
    rgx = re.search(
        "(?:https?:\/\/)?(?:m\.)?(?:www\.)?youtu\.?be(?:\.com)?\/?\S*(?:watch|embed)?(?:\S*v=|v\/|\/)([\w\-]+)(?:[\&\?]?([\w\-]+)?=?([\w\%\-]+)?)+",
        message
    )
    return rgx.group(0) if rgx is not None else None

In [16]:
df['youtube_url'] = df['message'].apply(find_youtube_url)

In [17]:
df_ytb_urls = df.dropna()
df_ytb_urls.head()

,message_id,date,message,youtube_url
0,66378,2023-03-04 22:40:51+00:00,https://youtu.be/CoJ23XNHgG0,https://youtu.be/CoJ23XNHgG0
1,66377,2023-03-04 22:32:39+00:00,https://youtu.be/kqNJaKRnuVg,https://youtu.be/kqNJaKRnuVg
5,66373,2023-03-04 22:27:39+00:00,https://youtu.be/bD6ifecX6rs,https://youtu.be/bD6ifecX6rs
65,66311,2023-02-25 05:36:46+00:00,🎵🇦🇷Loli Molina & Las Migas - Mensajes en el Ma...,https://youtu.be/n4T5BhDZon8
67,66309,2023-02-25 00:48:11+00:00,https://youtu.be/rMuTXcf3-6A,https://youtu.be/rMuTXcf3-6A


In [19]:
len(df_ytb_urls)

133

## Loop over existing youtube urls and get metadata from the `oembed` API

In [34]:
def get_youtube_oembed_info(video_url):
    oembed_url = f"https://www.youtube.com/oembed?url={video_url}&format=json"
    response = requests.get(oembed_url)
    if response.status_code != 200:
        return None
    ytb_json = response.json()
    sleep(0.3)
    return ytb_json['title'], ytb_json['author_name']

In [25]:
df_ytb_urls

,message_id,date,message,youtube_url
0,66378,2023-03-04 22:40:51+00:00,https://youtu.be/CoJ23XNHgG0,https://youtu.be/CoJ23XNHgG0
1,66377,2023-03-04 22:32:39+00:00,https://youtu.be/kqNJaKRnuVg,https://youtu.be/kqNJaKRnuVg
5,66373,2023-03-04 22:27:39+00:00,https://youtu.be/bD6ifecX6rs,https://youtu.be/bD6ifecX6rs
65,66311,2023-02-25 05:36:46+00:00,🎵🇦🇷Loli Molina & Las Migas - Mensajes en el Ma...,https://youtu.be/n4T5BhDZon8
67,66309,2023-02-25 00:48:11+00:00,https://youtu.be/rMuTXcf3-6A,https://youtu.be/rMuTXcf3-6A
...,...,...,...,...
932,66446,2023-03-05 14:54:14+00:00,https://youtu.be/TCUwG03ZPIw,https://youtu.be/TCUwG03ZPIw
933,66445,2023-03-05 14:43:08+00:00,https://youtu.be/dsvhwAVMU1A,https://youtu.be/dsvhwAVMU1A
937,66441,2023-03-05 14:32:21+00:00,https://youtu.be/bgMgYI1u8Mg,https://youtu.be/bgMgYI1u8Mg
939,66439,2023-03-05 14:27:54+00:00,https://youtu.be/GRKnMy17WxA,https://youtu.be/GRKnMy17WxA


In [35]:
get_youtube_oembed_info("https://youtu.be/CoJ23XNHgG0")

('Luedji Luna - Banho de folhas (clipe)', 'ybmusic')

In [36]:
df_ytb_urls['youtube_metadata'] = df_ytb_urls['youtube_url'].apply(get_youtube_oembed_info)

/tmp/ipykernel_6802/3206828650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ytb_urls['youtube_metadata'] = df_ytb_urls['youtube_url'].apply(get_youtube_oembed_info)


In [38]:
df_ytb_urls['youtube_metadata']

0       (Luedji Luna - Banho de folhas (clipe), ybmusic)
1      (SóCIRO - A Música Mais Fofa do Ano (Acústico)...
5      (Poesia Acústica #2 - Sobre Nós - Delacruz I M...
65     (Loli Molina & Las Migas - Mensajes en el Mar,...
67     (Joan Baez & Mercedes Sosa "Gracias A La Vida"...
                             ...                        
932              (Cássia Eller - De esquina, ahquesaco2)
933    (Ney Matogrosso, Pedro Luis E A Parede - Disri...
937                        (Nega Olívia, Bebeto - Topic)
939                        (Minha Preta, Bebeto - Topic)
947    (Toque de São Bento Grande de Angola - Paulo C...
Name: youtube_metadata, Length: 133, dtype: object

### Save dataset as .csv

This is meant to prevent rerunning the API calls.

In [96]:
df_ytb_urls.to_csv("df_ytb_urls.csv")

## Load artists from musicbrainz dump file

In order to replicate the artist-names file you will need to:
    
1. Access musicbrainz and download the artist.tar.xz dumpfile from <https://musicbrainz.org/doc/Development/JSON_Data_Dumps>
2. Filter the `artist` file and create a new `artist-names` file containing only artist names
    
        tar -xvf archive.tar.xz -C musicbrainz-artist-dump
        cd musicbrains-artist-dump/mbdump
        jq -r ". | .name" artist > artist-names

Note that you will need `jq` on your system to reproduce these steps. For installation see <https://stedolan.github.io/jq/>

**TODO:** We could have a much better matching system if we favored artists from specific regions over others. For example, we could create a set of all brazilian artists, a set of all US artists, english, french, etc based on popularity (meaning theres a bigger chance of an artist being from these specific countries in general).

We would then first search for matches on the brazillian set, then the US set, then the english set...

This can be done, roughly, by using the following filter:

```
cat artist | jq ". | select(.area.name == \"Brazil\")" > brazil-artists
cat artist | jq ". | select(.area.name == \"United States\")" > united-states-artists
```

**TODO:** What about repeating names? For example, "Rick Astley" would never be found, because there's a brazillian artist named "Rick".

Maybe we could save the assumed artist for each language?

In [49]:
!pip install unidecode

from unidecode import unidecode

  Using cached Unidecode-1.3.6-py3-none-any.whl (235 kB)



<div class="alert alert-block alert-warning">This method of cleaning the data will break many band names that are not strictly ascii (ex. russian, korean, etc)</div>


In [120]:
# removes accents and simbols, forces lowercase
def strip_func(line):
    if isinstance(line, tuple):
        line = " ".join(line)
    line = line.strip()
    #rm_symbols = [',', '!', '.', ';', '[', ']', ':', '*', '"', '-', '(', ')'] 
    line = unidecode(line).lower()
    return line#.translate({ord(x): '' for x in rm_symbols})

In [121]:
artist_names_file = os.path.join("musicbrainz-artist-dump/mbdump/brazil-artist-names")


with open(artist_names_file) as artist_file:
    artist_names = set(strip_func(line) for line in artist_file)

In [122]:
len(artist_names)

9684

In [123]:
artist_names

{'skema novo',
 'gustavo deppe',
 'carol celtico',
 'anisio mello junior',
 'polvo',
 'banex',
 'disforterror',
 'klaus ximenes',
 'leomarist',
 'night goat',
 'atomo',
 'galope rasante',
 'beline braga',
 'anderson leonardo',
 'alfredo gregorio de melo',
 'rdd',
 'zartrox',
 'hugo doche',
 'carlos de andrade',
 'mossa bildner',
 'olavo cavalheiro',
 'tokyo',
 'pedro ortaca',
 'amazon',
 'you can dance',
 'augusto olivani',
 'cury',
 'momento quatro',
 'absent',
 'biu do piseiro',
 'sound bullet',
 'carlos lima do espirito santo',
 'brasil 690',
 'toni goncalves',
 'marcelo cic',
 'leila diniz',
 'forrocacana',
 'midgard',
 'alexandre rabaco',
 'geo',
 'tritone',
 'otaviano romero',
 'semente cristal',
 'ernani bordinhao',
 'the divorce factory',
 'sychopaths',
 'banda mantiqueira',
 'ancientblood',
 'nelson goncalves',
 'janaynna',
 'vinicius apolonio rocha',
 'black oil',
 'alibi',
 'sonhador',
 'carlos imperial',
 'zico',
 'fausto nilo',
 'kid foguete',
 'robin jones',
 'terrorcidio

## Match youtube metadata to artist name by brute force

The following algorithm will loop over all word combinations in the youtube metadata (both video title and author) and try to match it to an existing artist name in the musicbrainz dump file.

In [103]:
df_ytb_urls = df_ytb_urls.dropna()
df_ytb_urls.head()

,message_id,date,message,youtube_url,youtube_metadata,spotify_api_query,spotify_api_return_status,spotify_api_return_json
0,66378,2023-03-04 22:40:51+00:00,https://youtu.be/CoJ23XNHgG0,https://youtu.be/CoJ23XNHgG0,luedji luna banho de folhas clipe ybmusic,luedji luna,,
1,66377,2023-03-04 22:32:39+00:00,https://youtu.be/kqNJaKRnuVg,https://youtu.be/kqNJaKRnuVg,sociro a musica mais fofa do ano acustico sociro,,,
5,66373,2023-03-04 22:27:39+00:00,https://youtu.be/bD6ifecX6rs,https://youtu.be/bD6ifecX6rs,poesia acustica #2 sobre nos delacruz i mari...,,,
65,66311,2023-02-25 05:36:46+00:00,🎵🇦🇷Loli Molina & Las Migas - Mensajes en el Ma...,https://youtu.be/n4T5BhDZon8,loli molina & las migas mensajes en el mar lo...,loli,,
67,66309,2023-02-25 00:48:11+00:00,https://youtu.be/rMuTXcf3-6A,https://youtu.be/rMuTXcf3-6A,joan baez & mercedes sosa gracias a la vida ro...,joan,,


In [104]:
df_ytb_urls['youtube_metadata'] = df_ytb_urls['youtube_metadata'].apply(strip_func)
df_ytb_urls.head()

,message_id,date,message,youtube_url,youtube_metadata,spotify_api_query,spotify_api_return_status,spotify_api_return_json
0,66378,2023-03-04 22:40:51+00:00,https://youtu.be/CoJ23XNHgG0,https://youtu.be/CoJ23XNHgG0,luedji luna banho de folhas clipe ybmusic,luedji luna,,
1,66377,2023-03-04 22:32:39+00:00,https://youtu.be/kqNJaKRnuVg,https://youtu.be/kqNJaKRnuVg,sociro a musica mais fofa do ano acustico sociro,,,
5,66373,2023-03-04 22:27:39+00:00,https://youtu.be/bD6ifecX6rs,https://youtu.be/bD6ifecX6rs,poesia acustica #2 sobre nos delacruz i mari...,,,
65,66311,2023-02-25 05:36:46+00:00,🎵🇦🇷Loli Molina & Las Migas - Mensajes en el Ma...,https://youtu.be/n4T5BhDZon8,loli molina & las migas mensajes en el mar lo...,loli,,
67,66309,2023-02-25 00:48:11+00:00,https://youtu.be/rMuTXcf3-6A,https://youtu.be/rMuTXcf3-6A,joan baez & mercedes sosa gracias a la vida ro...,joan,,


In [110]:
df_ytb_urls['spotify_api_query'] = ""
df_ytb_urls['spotify_api_return_status'] = ""
df_ytb_urls['spotify_api_return_json'] = ""
df_ytb_urls.query('spotify_api_query != ""')

,message_id,date,message,youtube_url,youtube_metadata,spotify_api_query,spotify_api_return_status,spotify_api_return_json


In [111]:
stopwords = {
    "o", "a", "os", "as", "e", "el", "the", "no", "na",
    "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11",
    "dvd"
}
stopwords = set(stopwords)

for metadata in df_ytb_urls['youtube_metadata']:
    assumed_artist_name = ""
    for word in metadata.split():
        assumed_artist_name = assumed_artist_name + " " + word
        assumed_artist_name = assumed_artist_name.strip()
        if assumed_artist_name in stopwords:
            continue
        if assumed_artist_name in artist_names:
            print(assumed_artist_name)
            df_ytb_urls.loc[df_ytb_urls['youtube_metadata'] == metadata, ['spotify_api_query']] = assumed_artist_name
            # validate against external API!!
            # find row where youtube_metadata == metadata,
            # save the value used in the api call,
            # the return status and the received json
            break

el efecto
leci brandao
rincon sapiencia
rincon sapiencia
rael
zeca baleiro
sant
maria bethania
clara nunes
elis regina
bia
el efecto
nando reis
alceu valenca
sepultura
thaide & dj hum
costa gold
organico
nando reis
organico
medulla
leandro
chimarruts
el efecto
el efecto
z'africa brasil
violeta de outono
kaatayra
criolo
bnegao
joelho de porco
rick
santana
o rappa
pitty
ney matogrosso
sandra
chico science
cassia eller
ney matogrosso


In [112]:
df_ytb_urls.query('spotify_api_query != ""')

,message_id,date,message,youtube_url,youtube_metadata,spotify_api_query,spotify_api_return_status,spotify_api_return_json
132,66244,2023-02-24 14:07:14+00:00,https://www.youtube.com/watch?v=mI9vuhcaNoM,https://www.youtube.com/watch?v=mI9vuhcaNoM,el efecto memorias do fogo 2018 disco comple...,el efecto,,
193,66183,2023-02-23 04:28:06+00:00,https://youtu.be/hSXIO0t4OlQ,https://youtu.be/hSXIO0t4OlQ,leci brandao ze do caroco radial by the orchard,leci brandao,,
250,66123,2023-02-19 14:04:05+00:00,https://youtu.be/Kwpb6zJqeKY,https://youtu.be/Kwpb6zJqeKY,rincon sapiencia coisas de brasil clipe part...,rincon sapiencia,,
253,66120,2023-02-19 13:57:12+00:00,https://youtu.be/vSVY1rzAW9w,https://youtu.be/vSVY1rzAW9w,rincon sapiencia a volta pra casa rincon sapi...,rincon sapiencia,,
254,66119,2023-02-19 12:34:51+00:00,https://youtu.be/C5f0P59kl2Y,https://youtu.be/C5f0P59kl2Y,rael do quintal session quem tem fe rael,rael,,
267,66105,2023-02-18 12:30:14+00:00,https://youtu.be/8dVwwKi5gsM,https://youtu.be/8dVwwKi5gsM,zeca baleiro telegrama baladas do asfalto & o...,zeca baleiro,,
387,65982,2023-02-12 16:23:07+00:00,https://youtu.be/XGNb0oBSTtg,https://youtu.be/XGNb0oBSTtg,sant nuvem negra o mundo ao norte,sant,,
397,65972,2023-02-12 13:18:49+00:00,https://youtu.be/YPO1iaetL2I,https://youtu.be/YPO1iaetL2I,maria bethania reconvexo ao vivo amor festa ...,maria bethania,,
398,65971,2023-02-12 13:14:05+00:00,https://youtu.be/bO_7pj92hww,https://youtu.be/bO_7pj92hww,clara nunes partido alto com clementina clara ...,clara nunes,,
399,65970,2023-02-12 13:11:01+00:00,https://youtu.be/caEFyFRc91c,https://youtu.be/caEFyFRc91c,elis regina & adoniran barbosa tiro ao alvaro...,elis regina,,
